In [2]:
# test if the clusters are running

from ipyparallel import Client
rc = Client()
print("Number of clusters running:", len(rc))
dview = rc[:]

Number of clusters running: 3


Import necessary libraries at once

In [3]:
import boto3
import smart_open
import ast
import pandas as pd
from collections import Counter

Collect all the keys to objects stored in the S3 bucket

In [4]:
s3_resource = boto3.resource('s3')
sparkify3_logs = s3_resource.Bucket('sparkify3-logs')
all_keys = []

for obj in sparkify3_logs.objects.all():
    all_keys.append(obj.key)
    
print("Number of json objects found:", len(all_keys))

Number of json objects found: 30


Scatter keys

In [6]:
%%px

import boto3
import json
from collections import Counter
import smart_open
import ast

def counter_test1(keys):
    s3_client = boto3.client('s3')
    bucket = 'sparkify3-logs'
    artistCounter = {}
    songCounter = {}
    
    for key in keys:
#         obj = s3_client.get_object(Bucket=bucket, Key=str(key))
#         obj = json.loads(obj['Body'].read())
        for line in smart_open.smart_open('s3://{}/{}'.format(bucket, key)):
            try:
                dict_line = ast.literal_eval(line.decode("utf-8"))
                try:
                    # skip None values
                    if dict_line['artist']:
                        artistCounter[str(dict_line['artist'])] += 1
                except:
                    artistCounter[str(dict_line['artist'])] = 1

                # repeat for songs
                try:
                    if dict_line['song']:
                        songCounter[str(dict_line['song'])] += 1
                except:
                    songCounter[str(dict_line['song'])] = 1
            except:
                pass
            
    return Counter(artistCounter), Counter(songCounter)
    

In [7]:
import time
start = time.perf_counter()
dview.scatter('keys', all_keys)
%px y = [counter_test1(keys)]
y = dview.gather('y')

print("Time taken to get all results = {:.4f}s".format(time.perf_counter() - start))

%px:   0%|          | 0/3 [00:00<?, ?tasks/s]

Time taken to get all results = 4.8017s


In [8]:
artistCounter = Counter({})
songCounter = Counter({})

for (artists, songs) in y:
    artistCounter += artists
    songCounter += songs
    
artistCounter = pd.DataFrame(artistCounter.most_common(10),
                            columns = ['Artist Name', 'Count'], index=range(1,11))

songCounter = pd.DataFrame(songCounter.most_common(10),
                            columns = ['Song Name', 'Count'], index=range(1,11))



In [9]:
pd.options.display.width = 0
print("Top 10 Artists")
display(artistCounter)

print("\n\nTop 10 Songs")
display(songCounter)

Top 10 Artists


,Artist Name,Count
1,Coldplay,58
2,Kings Of Leon,55
3,Dwight Yoakam,38
4,The Black Keys,36
5,Florence + The Machine,35
6,Jack Johnson,35
7,Muse,35
8,BjÃÂ¶rk,33
9,The Killers,31
10,John Mayer,31




Top 10 Songs


,Song Name,Count
1,You're The One,37
2,Undo,28
3,Revelry,27
4,Sehr kosmisch,21
5,Horn Concerto No. 4 in E flat K495: II. Romanc...,19
6,Secrets,17
7,Canada,17
8,Dog Days Are Over (Radio Edit),16
9,ReprÃÂ©sente,14
10,Invalid,14
